# 1 load data and get prediction result

In [1]:
import demo_load_la
import torch
engine = demo_load_la.get_engine()
#create model

engine.model.load_state_dict(torch.load("garage/metr-la_best_2.7_2024.pth"))
#load param

yhat=demo_load_la.get_time_T(engine)
#get predict result

engine get!


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
yhat.shape


torch.Size([6850, 207, 12])

# 2  read raw METR-LA

In [4]:
import h5py
file='data/METR-LA/metr-la.h5'
 
f=h5py.File(file,'r')
f.keys()

<KeysViewHDF5 ['df']>

In [5]:
f['df']

<HDF5 group "/df" (4 members)>

In [6]:
f['df'].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values']>

## 2.1 time for metr-la

In [7]:
f['df']['axis1'][:].shape,f['df']['axis1'][:]

((34272,),
 array([1330560000000000000, 1330560300000000000, 1330560600000000000, ...,
        1340840700000000000, 1340841000000000000, 1340841300000000000]))

In [8]:
tst_time=f['df']['axis1'][-10419:]
tst_time.shape

(10419,)

In [9]:
import pandas as pd
 
datetimes=pd.to_datetime(tst_time)
datetimes

DatetimeIndex(['2012-05-22 19:45:00', '2012-05-22 19:50:00',
               '2012-05-22 19:55:00', '2012-05-22 20:00:00',
               '2012-05-22 20:05:00', '2012-05-22 20:10:00',
               '2012-05-22 20:15:00', '2012-05-22 20:20:00',
               '2012-05-22 20:25:00', '2012-05-22 20:30:00',
               ...
               '2012-06-27 23:10:00', '2012-06-27 23:15:00',
               '2012-06-27 23:20:00', '2012-06-27 23:25:00',
               '2012-06-27 23:30:00', '2012-06-27 23:35:00',
               '2012-06-27 23:40:00', '2012-06-27 23:45:00',
               '2012-06-27 23:50:00', '2012-06-27 23:55:00'],
              dtype='datetime64[ns]', length=10419, freq=None)

## 2.2 raw value of metr-la

In [10]:
speed=f['df']['block0_values'][:]

In [11]:
speed

array([[64.375     , 67.625     , 67.125     , ..., 59.25      ,
        69.        , 61.875     ],
       [62.66666667, 68.55555556, 65.44444444, ..., 55.88888889,
        68.44444444, 62.875     ],
       [64.        , 63.75      , 60.        , ..., 61.375     ,
        69.85714286, 62.        ],
       ...,
       [67.        , 59.66666667, 69.55555556, ..., 63.33333333,
        67.44444444, 61.22222222],
       [66.75      , 62.25      , 66.        , ..., 63.        ,
        67.875     , 63.5       ],
       [65.11111111, 66.88888889, 66.77777778, ..., 63.55555556,
        68.66666667, 61.77777778]])

In [12]:
mean,std=speed.mean(),speed.std()
mean,std

(53.7190211024135, 20.261430789619187)

# 3 metr-la location

In [13]:
import pandas as pd
df=pd.read_csv('data/METR-LA/graph_sensor_locations.csv',names=['station_id','lat','lon'], skiprows=1)
df

,station_id,lat,lon
0,773869,34.15497,-118.31829
1,767541,34.11621,-118.23799
2,767542,34.11641,-118.23819
3,717447,34.07248,-118.26772
4,717446,34.07142,-118.26572
...,...,...,...
202,717592,34.14604,-118.22430
203,717595,34.14163,-118.18290
204,772168,34.16542,-118.47985
205,718141,34.15133,-118.37456


In [14]:
df[df['station_id'].isin([717492,773939,765604,768066])]

,station_id,lat,lon
10,765604,34.16415,-118.38223
40,768066,34.15118,-118.37480
61,773939,34.15297,-118.37226
72,717492,34.15459,-118.37935


In [15]:
mean_lat=df['lat'].mean()
mean_lon=df['lon'].mean()
mean_lat,mean_lon

(34.13685429951691, -118.32048154589373)

In [31]:
import folium
 
 
m=folium.Map(location=(mean_lat,mean_lon),zoom_start=12)
 
for i in df.iterrows():
    #print(i)
    tmp_lat=i[1]['lat']
    tmp_lon=i[1]['lon']
    tmp_id=i[1]['station_id']
    #print(tmp_lat,tmp_lon)
    folium.Marker(location=(tmp_lat,tmp_lon),
                  icon=folium.Icon(icon='car',
                                   prefix='fa',
                                  color='green',
                                  icon_color='blue'),
                 popup=str(int(tmp_id))+' : '+str(tmp_lat)+' , '+str(tmp_lon))#.add_to(m)
m

In [16]:
import folium
 
 
m=folium.Map(location=(mean_lat,mean_lon),zoom_start=12)
 
for i in df.iterrows():
    #print(i)
    tmp_lat=i[1]['lat']
    tmp_lon=i[1]['lon']
    tmp_id=i[1]['station_id']
    #print(tmp_lat,tmp_lon)
    folium.Marker(location=(tmp_lat,tmp_lon),
                  icon=folium.Icon(icon='car',
                                   prefix='fa',
                                  color='green',
                                  icon_color='blue'),
                 popup=str(int(tmp_id))+' : '+str(tmp_lat)+' , '+str(tmp_lon)).add_to(m)
m

# 4 get a time slot's predicted speed

index 0 refers to 2012-05-22 19:45:00, index i refers to the ith 5min from index 0

In [19]:
num=int(input('input a number, from 0 to 10418'))

input a number, from 0 to 10418  1234


In [20]:
datetimes[num]

Timestamp('2012-05-27 02:35:00')

torch.Size([6850, 207])

In [21]:
pred=yhat[num,:,0].detach().cpu()
pred=pred*std+mean
pred=pred.tolist()
pred

[64.96292877197266,
 65.1851577758789,
 68.26103973388672,
 51.840084075927734,
 34.719398498535156,
 57.18785858154297,
 62.34626770019531,
 64.6636962890625,
 61.08518600463867,
 27.537033081054688,
 64.56204986572266,
 63.542240142822266,
 16.93670654296875,
 66.72808837890625,
 58.78302001953125,
 34.45085525512695,
 30.596546173095703,
 58.15584945678711,
 55.645503997802734,
 61.75784683227539,
 67.08619689941406,
 60.409584045410156,
 48.46856689453125,
 57.630897521972656,
 66.11466217041016,
 64.52442169189453,
 53.702327728271484,
 62.881378173828125,
 69.06324768066406,
 44.010345458984375,
 60.13242721557617,
 66.15966796875,
 65.21377563476562,
 55.401336669921875,
 66.88298034667969,
 62.72294235229492,
 67.12538146972656,
 67.00454711914062,
 60.76796340942383,
 58.973243713378906,
 65.32231903076172,
 62.968746185302734,
 65.51766967773438,
 24.403230667114258,
 67.54029846191406,
 67.01317596435547,
 61.78092575073242,
 56.732330322265625,
 55.29063415527344,
 60.61661

In [22]:
import folium
 
 
m=folium.Map(location=(mean_lat,mean_lon),zoom_start=12)
 
for i in range(len(df)):
    #print(i)
    tmp_lat=df.at[i,'lat']
    tmp_lon=df.at[i,'lon']
    tmp_id=df.at[i,'station_id']
    #print(tmp_lat,tmp_lon)
    folium.Marker(location=(tmp_lat,tmp_lon),
                  icon=folium.Icon(icon='car',
                                   prefix='fa',
                                  color='green',
                                  icon_color='blue'),
                 popup='id: '+str(int(tmp_id))+' speed: '+str(pred[i])).add_to(m)
m

# 6 更新版：如何计算转向概率

## 6.1 速度差

**速度差**指的是驶出路段和驶入路段之间的速度差异。

影响：

- **速度差正值较大**（即驶入路段速度明显高于驶出路段）：这通常意味着驶入路段比驶出路段拥堵程度低，流通性更好。在这种情况下，司机可能更倾向于转向这个流畅的路段，因此**转向概率增加**。

- **速度差负值较大**（即驶入路段速度低于驶出路段）：这表明驶入路段比驶出路段更拥堵。因此，司机可能会避免转入这个路段，**转向概率减少**。

## 6.2 驶入路段速度 

**驶入路段速度**指的是潜在转向路段的速度。

影响：
- **驶入路段速度较高**：这通常表示该路段流通性好，拥堵程度低。司机可能倾向于转向这样的路段，以避免拥堵，因此**转向这个路段的概率增加**。
- **驶入路段速度较低**：如果潜在的转向路段速度低，表明该路段可能较为拥堵。在这种情况下，司机可能会避免转入这个路段，**转向概率减少**。

所以更新后的 吸引函数为：

$P=\frac{1}{1+e^{-(a \cdot \Delta V+b \cdot \sqrt{V_{in})}}}$

- $P$ 是从驶出路段转向驶入路段的概率
- $\Delta V=V_{in}-V_{out}$ 是速度差
- $V_{in}$是驶入路段（经过路口之后）的速度

这个函数模型通过结合速度差、驶出路段速度、和驶入路段速度的影响，提供了一个综合的视角来预测司机的转向行为。它采用逻辑斯蒂形式来确保预测的转向概率始终在0到1之间，并能够捕捉到这些因素的非线性和相互作用效应。通过调整系数 a，b，c，可以根据具体的交通环境和司机行为模式来调整模型。

同样地，通过这种方式得到某一驶出路段的各个驶入路段的吸引值 P1 P2 P3，然后我们使用softmax(P1,P2,P3)得到对应的概率

### 优势 - 效果方面

- 捕捉复杂交通动态：通过融合速度差、驶出路段速度、和驶入路段速度，这种方法能够更全面地理解和预测司机的行为，尤其是在复杂的交通环境中。
- 灵活适应多变交通状况：参数化的模型允许根据不同的交通环境和司机行为模式进行调整，提供高度的适应性。
- 更好的预测准确性：逻辑斯蒂形式的函数确保了预测值在合理的范围内（即0到1之间），避免了不现实的概率估计。
- 非线性特性的体现：这种模型能够表现出交通流中的非线性特性，如在高速路段小的速度变化可能不会显著影响司机行为，而在低速路段则可能有显著影响。

# 优势 - 效率方面
- 计算效率：虽然模型包含非线性元素，但其计算依然可以保持高效，特别是当使用现代计算工具和优化算法时。
- 易于实现和集成：这种模型的结构相对简单，可以容易地在各种交通模拟软件中实现和集成。
- 适用于实时或近实时分析：由于计算效率高，这种方法适合用于实时或近实时的交通流分析，有助于交通管理和控制。

# 参数a，b，c的考量

## a
在高速公路上，驶入路段的速度如果明显高于驶出路段，这对驾驶者来说是一个强烈的吸引因素，因为在高速行驶条件下，即使是小的速度提升也可以显著减少旅行时间。

所以a相对较大，以反映速度差在高速公路上对转向决策的重要性

## b
考虑到高速公路上速度的变化范围相对较小，b 的值可能不需要像 a 那么高

## c
c 的值也应该相对小一些，以考虑到即使在高速条件下，驾驶者可能仍然会为了更高的速度或更好的路况做出转向决策

![](metr-la.png)

In [23]:
a=1
b=0.05
def func_prob(v_in,v_out):
    #v_in is a float num, v_out is a tensor
    v_out_expanded = v_out.unsqueeze(1).expand_as(v_in)
    delta_v=v_in-v_out_expanded 
    #print(a*delta_v,b* torch.sqrt(v_in))
    prob=1/(1+torch.exp(-(a*delta_v+b* torch.sqrt(v_in))))
    #print(prob)
    #print(a*delta_v,c*v_in_expanded)
    return torch.softmax(prob,dim=1)

v_out=torch.tensor([62.56])
v_in=torch.tensor([[65.72,62.63,32.36]])
#直行 左转 右转
func_prob(v_in,v_out)

tensor([[0.4814, 0.3365, 0.1820]])

In [24]:
pred_out=yhat[:,72,0].detach().cpu()
#717492
pred_out=pred_out*std+mean
pred_out.shape

torch.Size([6850])

In [25]:
pred_straight=yhat[:,61,0].detach().cpu()
#773939
pred_left=yhat[:,10,0].detach().cpu()
#765604
pred_right=yhat[:,40,0].detach().cpu()
#7680

pred_straight=pred_straight*std+mean
pred_left=pred_left*std+mean
pred_right=pred_right*std+mean
pred_straight[:5]

tensor([56.3108, 57.1304, 53.8553, 54.1186, 54.2205])

In [26]:
torch.stack((pred_straight,pred_left,pred_right),dim=1)

tensor([[56.3108, 49.1846, 64.0034],
        [57.1304, 52.0076, 64.6406],
        [53.8553, 44.5544, 64.3854],
        ...,
        [65.3668, 61.2818, 66.6989],
        [64.7760, 59.3089, 67.2790],
        [65.1004, 59.4265, 67.7208]])

In [27]:
res_tensor_prob=func_prob(torch.stack((pred_straight,pred_left,pred_right),dim=1),pred_out)
res_tensor_prob

tensor([[0.2622, 0.1986, 0.5392],
        [0.2514, 0.2018, 0.5467],
        [0.2138, 0.2119, 0.5743],
        ...,
        [0.3907, 0.1809, 0.4284],
        [0.3991, 0.1673, 0.4336],
        [0.3598, 0.1767, 0.4635]])

In [28]:
datetimes.shape

(10419,)

In [29]:
turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'left', 'right'])
turning_prob


,straight,left,right
2012-06-04 05:10:00,0.262208,0.198634,0.539158
2012-06-04 05:15:00,0.251417,0.201846,0.546737
2012-06-04 05:20:00,0.213768,0.211900,0.574332
2012-06-04 05:25:00,0.217847,0.210526,0.571627
2012-06-04 05:30:00,0.217284,0.210639,0.572077
...,...,...,...
2012-06-27 23:35:00,0.396154,0.203375,0.400471
2012-06-27 23:40:00,0.395678,0.179543,0.424779
2012-06-27 23:45:00,0.390704,0.180900,0.428396
2012-06-27 23:50:00,0.399098,0.167330,0.433572


In [30]:
turning_prob.describe()

,straight,left,right
count,6844.000000,6844.000000,6844.000000
mean,0.309771,0.312466,0.377764
std,0.105247,0.097524,0.088715
min,0.155362,0.155363,0.155362
25%,0.211911,0.227548,0.336505
50%,0.333348,0.316577,0.402387
75%,0.395381,0.417503,0.422319
max,0.574571,0.576117,0.576117


![](metrla-1.png)

In [33]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [769926,769847,769941,769831,767610,767609]):
            print(tmp_id,i)

769941 35
767609 46
769831 50
767610 66
769847 121
769926 189


## for 767609

In [47]:
pred_right=yhat[:,50,0].detach().cpu()
#773939
pred_left=yhat[:,189,0].detach().cpu()
#7680

pred_out=yhat[:,46,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean


res_tensor_prob=func_prob(torch.stack((pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['right', 'left'])
f='METRLA_res/'+'767609'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,right,left
0,2012-06-04 05:10:00,0.501306,0.498694
1,2012-06-04 05:15:00,0.502462,0.497538
2,2012-06-04 05:20:00,0.485467,0.514533
3,2012-06-04 05:25:00,0.460840,0.539160
4,2012-06-04 05:30:00,0.470657,0.529343
...,...,...,...
6845,2012-06-27 23:35:00,0.289835,0.710165
6846,2012-06-27 23:40:00,0.283172,0.716828
6847,2012-06-27 23:45:00,0.303278,0.696722
6848,2012-06-27 23:50:00,0.271082,0.728918


## for 769941

In [46]:
pred_straight=yhat[:,189,0].detach().cpu()
#773939
pred_left=yhat[:,66,0].detach().cpu()
#7680

pred_out=yhat[:,35,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_left=pred_left*std+mean


res_tensor_prob=func_prob(torch.stack((pred_straight,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'left'])
turning_prob.to_csv('METRLA_res/'+'769941'+'_metr_la.csv')
pd.read_csv('METRLA_res/'+'769941'+'_metr_la.csv')

,Unnamed: 0,straight,left
0,2012-06-04 05:10:00,0.371794,0.628206
1,2012-06-04 05:15:00,0.454282,0.545718
2,2012-06-04 05:20:00,0.480591,0.519409
3,2012-06-04 05:25:00,0.478699,0.521301
4,2012-06-04 05:30:00,0.436979,0.563021
...,...,...,...
6845,2012-06-27 23:35:00,0.728381,0.271619
6846,2012-06-27 23:40:00,0.497200,0.502800
6847,2012-06-27 23:45:00,0.496647,0.503353
6848,2012-06-27 23:50:00,0.499136,0.500864


## for 769847

In [44]:
pred_straight=yhat[:,50,0].detach().cpu()
#773939
pred_right=yhat[:,66,0].detach().cpu()
#7680

pred_out=yhat[:,121,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean


res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right'])
turning_prob.to_csv('METRLA_res/'+'769847'+'_metr_la.csv')

In [45]:
pd.read_csv('METRLA_res/'+'769847'+'_metr_la.csv')

,Unnamed: 0,straight,right
0,2012-06-04 05:10:00,0.328871,0.671129
1,2012-06-04 05:15:00,0.361363,0.638637
2,2012-06-04 05:20:00,0.321348,0.678652
3,2012-06-04 05:25:00,0.346910,0.653090
4,2012-06-04 05:30:00,0.375966,0.624034
...,...,...,...
6845,2012-06-27 23:35:00,0.628175,0.371825
6846,2012-06-27 23:40:00,0.308678,0.691322
6847,2012-06-27 23:45:00,0.371193,0.628807
6848,2012-06-27 23:50:00,0.269339,0.730661


![](metrla-2.png)

In [48]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [717590,717587,767471,767470,773996,773995,767621,767620]):
            print(tmp_id,i)

767620 7
767471 11
767470 55
767621 85
773996 152
773995 153
717587 171
717590 201


## for  767471

In [52]:
pred_straight=yhat[:,85,0].detach().cpu()
#773939
pred_right=yhat[:,201,0].detach().cpu()
#7680
pred_left=yhat[:,153,0].detach().cpu()

pred_out=yhat[:,11,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'767471'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.529892,0.253537,0.216572
1,2012-06-04 05:15:00,0.552253,0.231502,0.216245
2,2012-06-04 05:20:00,0.528859,0.262733,0.208408
3,2012-06-04 05:25:00,0.510057,0.288437,0.201506
4,2012-06-04 05:30:00,0.445111,0.332338,0.222551
...,...,...,...,...
6845,2012-06-27 23:35:00,0.374919,0.366240,0.258842
6846,2012-06-27 23:40:00,0.333334,0.333334,0.333332
6847,2012-06-27 23:45:00,0.345798,0.344578,0.309624
6848,2012-06-27 23:50:00,0.337669,0.336248,0.326083


## for 773996

In [51]:
pred_straight=yhat[:,201,0].detach().cpu()
#773939
pred_right=yhat[:,55,0].detach().cpu()
#7680
pred_left=yhat[:,85,0].detach().cpu()

pred_out=yhat[:,152,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'773996'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.339955,0.317521,0.342524
1,2012-06-04 05:15:00,0.350459,0.286890,0.362651
2,2012-06-04 05:20:00,0.338748,0.316456,0.344796
3,2012-06-04 05:25:00,0.387183,0.219446,0.393370
4,2012-06-04 05:30:00,0.367414,0.261091,0.371495
...,...,...,...,...
6845,2012-06-27 23:35:00,0.415575,0.163816,0.420608
6846,2012-06-27 23:40:00,0.409084,0.178397,0.412520
6847,2012-06-27 23:45:00,0.412202,0.174175,0.413624
6848,2012-06-27 23:50:00,0.372991,0.253650,0.373359


## for 767620

In [50]:
pred_straight=yhat[:,55,0].detach().cpu()
#773939
pred_right=yhat[:,153,0].detach().cpu()
#7680
pred_left=yhat[:,201,0].detach().cpu()

pred_out=yhat[:,7,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'767620'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.294071,0.318505,0.387424
1,2012-06-04 05:15:00,0.295026,0.323656,0.381318
2,2012-06-04 05:20:00,0.292702,0.276400,0.430898
3,2012-06-04 05:25:00,0.232951,0.280167,0.486882
4,2012-06-04 05:30:00,0.221029,0.319434,0.459537
...,...,...,...,...
6845,2012-06-27 23:35:00,0.187833,0.335679,0.476488
6846,2012-06-27 23:40:00,0.200368,0.316741,0.482891
6847,2012-06-27 23:45:00,0.189671,0.327973,0.482356
6848,2012-06-27 23:50:00,0.192235,0.336299,0.471466


## for 717587

In [49]:
pred_straight=yhat[:,153,0].detach().cpu()
#773939
pred_right=yhat[:,85,0].detach().cpu()
#7680
pred_left=yhat[:,55,0].detach().cpu()

pred_out=yhat[:,171,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'717587'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.225886,0.559030,0.215084
1,2012-06-04 05:15:00,0.226330,0.559968,0.213702
2,2012-06-04 05:20:00,0.220448,0.554136,0.225416
3,2012-06-04 05:25:00,0.224137,0.563656,0.212207
4,2012-06-04 05:30:00,0.248011,0.539512,0.212477
...,...,...,...,...
6845,2012-06-27 23:35:00,0.266613,0.482524,0.250863
6846,2012-06-27 23:40:00,0.251141,0.514995,0.233864
6847,2012-06-27 23:45:00,0.271110,0.468794,0.260096
6848,2012-06-27 23:50:00,0.261062,0.482489,0.256449


![](metrla-3.png)

In [53]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [773953,773954,718379,761599,717578,716554,759772,716968]):
            print(tmp_id,i)

717578 24
716554 53
773953 54
773954 58
718379 95
761599 109
716968 112
759772 194


## for 759772

In [57]:
pred_straight=yhat[:,95,0].detach().cpu()
#773939
pred_right=yhat[:,58,0].detach().cpu()
#7680
pred_left=yhat[:,24,0].detach().cpu()

pred_out=yhat[:,194,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'759772'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.333428,0.333139,0.333434
1,2012-06-04 05:15:00,0.333573,0.332759,0.333668
2,2012-06-04 05:20:00,0.333410,0.333171,0.333419
3,2012-06-04 05:25:00,0.333415,0.333055,0.333530
4,2012-06-04 05:30:00,0.334297,0.330713,0.334990
...,...,...,...,...
6845,2012-06-27 23:35:00,0.330724,0.334523,0.334753
6846,2012-06-27 23:40:00,0.322203,0.338822,0.338975
6847,2012-06-27 23:45:00,0.331266,0.334284,0.334449
6848,2012-06-27 23:50:00,0.332392,0.333803,0.333805


## for  716554

In [56]:
pred_straight=yhat[:,58,0].detach().cpu()
#773939
pred_right=yhat[:,112,0].detach().cpu()
#7680
pred_left=yhat[:,95,0].detach().cpu()

pred_out=yhat[:,53,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'716554'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.331734,0.333768,0.334498
1,2012-06-04 05:15:00,0.334926,0.328286,0.336788
2,2012-06-04 05:20:00,0.333904,0.330336,0.335760
3,2012-06-04 05:25:00,0.336297,0.326510,0.337193
4,2012-06-04 05:30:00,0.333000,0.333247,0.333753
...,...,...,...,...
6845,2012-06-27 23:35:00,0.340655,0.331244,0.328101
6846,2012-06-27 23:40:00,0.360111,0.349411,0.290477
6847,2012-06-27 23:45:00,0.353165,0.327373,0.319462
6848,2012-06-27 23:50:00,0.333335,0.333333,0.333332


##  for 761599

In [55]:
pred_straight=yhat[:,112,0].detach().cpu()
#773939
pred_right=yhat[:,24,0].detach().cpu()
#7680
pred_left=yhat[:,58,0].detach().cpu()

pred_out=yhat[:,109,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'761599'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.333945,0.335114,0.330941
1,2012-06-04 05:15:00,0.277954,0.377791,0.344255
2,2012-06-04 05:20:00,0.323593,0.341455,0.334952
3,2012-06-04 05:25:00,0.297220,0.355233,0.347547
4,2012-06-04 05:30:00,0.330205,0.344619,0.325176
...,...,...,...,...
6845,2012-06-27 23:35:00,0.324303,0.338407,0.337290
6846,2012-06-27 23:40:00,0.316247,0.342920,0.340834
6847,2012-06-27 23:45:00,0.322109,0.339587,0.338304
6848,2012-06-27 23:50:00,0.330166,0.334923,0.334911


## for 773953

In [54]:
pred_straight=yhat[:,24,0].detach().cpu()
#773939
pred_right=yhat[:,112,0].detach().cpu()
#7680
pred_left=yhat[:,95,0].detach().cpu()

pred_out=yhat[:,54,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'773953'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.539451,0.205290,0.255260
1,2012-06-04 05:15:00,0.562675,0.210635,0.226690
2,2012-06-04 05:20:00,0.538678,0.203338,0.257984
3,2012-06-04 05:25:00,0.563431,0.211805,0.224765
4,2012-06-04 05:30:00,0.528595,0.212023,0.259382
...,...,...,...,...
6845,2012-06-27 23:35:00,0.505673,0.254160,0.240167
6846,2012-06-27 23:40:00,0.525173,0.269732,0.205096
6847,2012-06-27 23:45:00,0.534361,0.237359,0.228279
6848,2012-06-27 23:50:00,0.571536,0.214629,0.213835


![](metrla-4.png)
765604

In [59]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [717493,768066,718141,717570,717571,765604,764101,769467]):
            print(tmp_id,i)

765604 10
768066 40
717493 73
769467 113
717571 117
717570 118
764101 176
718141 205


## for 764101

In [63]:
pred_straight=yhat[:,40,0].detach().cpu()
#773939
pred_right=yhat[:,113,0].detach().cpu()
#7680
pred_left=yhat[:,118,0].detach().cpu()

pred_out=yhat[:,176,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'764101'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.348918,0.314318,0.336764
1,2012-06-04 05:15:00,0.357553,0.303423,0.339025
2,2012-06-04 05:20:00,0.352830,0.304091,0.343078
3,2012-06-04 05:25:00,0.347948,0.309917,0.342135
4,2012-06-04 05:30:00,0.350997,0.281758,0.367245
...,...,...,...,...
6845,2012-06-27 23:35:00,0.368597,0.287240,0.344163
6846,2012-06-27 23:40:00,0.335853,0.292592,0.371555
6847,2012-06-27 23:45:00,0.345543,0.290954,0.363504
6848,2012-06-27 23:50:00,0.355775,0.308398,0.335826


## for 717571

In [62]:
pred_straight=yhat[:,113,0].detach().cpu()
#773939
pred_right=yhat[:,10,0].detach().cpu()
#7680
pred_left=yhat[:,40,0].detach().cpu()

pred_out=yhat[:,117,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'717571'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.251444,0.251286,0.497271
1,2012-06-04 05:15:00,0.236424,0.235884,0.527691
2,2012-06-04 05:20:00,0.273879,0.273649,0.452472
3,2012-06-04 05:25:00,0.266054,0.263704,0.470242
4,2012-06-04 05:30:00,0.283929,0.283607,0.432464
...,...,...,...,...
6845,2012-06-27 23:35:00,0.287352,0.264006,0.448642
6846,2012-06-27 23:40:00,0.301851,0.290267,0.407882
6847,2012-06-27 23:45:00,0.291134,0.270319,0.438548
6848,2012-06-27 23:50:00,0.263514,0.259608,0.476878


## for 718141

In [61]:
pred_straight=yhat[:,10,0].detach().cpu()
#773939
pred_right=yhat[:,118,0].detach().cpu()
#7680
pred_left=yhat[:,113,0].detach().cpu()

pred_out=yhat[:,205,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'718141'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.261466,0.476887,0.261647
1,2012-06-04 05:15:00,0.259361,0.480912,0.259727
2,2012-06-04 05:20:00,0.259989,0.479588,0.260423
3,2012-06-04 05:25:00,0.248094,0.499365,0.252542
4,2012-06-04 05:30:00,0.229872,0.538660,0.231468
...,...,...,...,...
6845,2012-06-27 23:35:00,0.280122,0.418355,0.301523
6846,2012-06-27 23:40:00,0.249948,0.481010,0.269042
6847,2012-06-27 23:45:00,0.263029,0.454547,0.282423
6848,2012-06-27 23:50:00,0.280239,0.435707,0.284054


## for 717493

In [60]:
pred_straight=yhat[:,118,0].detach().cpu()
#773939
pred_right=yhat[:,40,0].detach().cpu()
#7680
pred_left=yhat[:,10,0].detach().cpu()

pred_out=yhat[:,73,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'717493'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.418649,0.422913,0.158438
1,2012-06-04 05:15:00,0.421736,0.422322,0.155942
2,2012-06-04 05:20:00,0.421784,0.422248,0.155967
3,2012-06-04 05:25:00,0.421384,0.422050,0.156565
4,2012-06-04 05:30:00,0.422307,0.422227,0.155466
...,...,...,...,...
6845,2012-06-27 23:35:00,0.396029,0.397506,0.206466
6846,2012-06-27 23:40:00,0.418016,0.413368,0.168615
6847,2012-06-27 23:45:00,0.406273,0.404729,0.188999
6848,2012-06-27 23:50:00,0.410268,0.410801,0.178931


![](metrla-5.png)

In [64]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [772167,717816,717804,760024,764794,717818,765171,769467]):
            print(tmp_id,i)

717816 9
717804 26
760024 43
765171 77
717818 104
769467 113
764794 158
772167 197


## for 765171

In [68]:
pred_straight=yhat[:,9,0].detach().cpu()
#773939
pred_right=yhat[:,113,0].detach().cpu()
#7680
pred_left=yhat[:,43,0].detach().cpu()

pred_out=yhat[:,77,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'765171'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.469288,0.174307,0.356405
1,2012-06-04 05:15:00,0.435040,0.174376,0.390583
2,2012-06-04 05:20:00,0.434845,0.169618,0.395536
3,2012-06-04 05:25:00,0.492854,0.197940,0.309206
4,2012-06-04 05:30:00,0.456865,0.174037,0.369097
...,...,...,...,...
6845,2012-06-27 23:35:00,0.334645,0.332412,0.332943
6846,2012-06-27 23:40:00,0.338195,0.330856,0.330950
6847,2012-06-27 23:45:00,0.334450,0.332328,0.333222
6848,2012-06-27 23:50:00,0.333339,0.333324,0.333337


## for 764794

In [67]:
pred_straight=yhat[:,113,0].detach().cpu()
#773939
pred_right=yhat[:,104,0].detach().cpu()
#7680
pred_left=yhat[:,9,0].detach().cpu()

pred_out=yhat[:,158,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'764794'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.192162,0.400383,0.407455
1,2012-06-04 05:15:00,0.211595,0.385387,0.403017
2,2012-06-04 05:20:00,0.224128,0.386218,0.389654
3,2012-06-04 05:25:00,0.292706,0.350940,0.356354
4,2012-06-04 05:30:00,0.203924,0.391409,0.404667
...,...,...,...,...
6845,2012-06-27 23:35:00,0.273279,0.322855,0.403866
6846,2012-06-27 23:40:00,0.259433,0.338542,0.402025
6847,2012-06-27 23:45:00,0.269895,0.324171,0.405934
6848,2012-06-27 23:50:00,0.282165,0.317168,0.400667


## for 717804

In [66]:
pred_straight=yhat[:,104,0].detach().cpu()
#773939
pred_right=yhat[:,43,0].detach().cpu()
#7680
pred_left=yhat[:,113,0].detach().cpu()

pred_out=yhat[:,26,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'717804'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.404322,0.424043,0.171634
1,2012-06-04 05:15:00,0.389781,0.420885,0.189334
2,2012-06-04 05:20:00,0.404453,0.405591,0.189956
3,2012-06-04 05:25:00,0.366470,0.364026,0.269504
4,2012-06-04 05:30:00,0.392734,0.398621,0.208645
...,...,...,...,...
6845,2012-06-27 23:35:00,0.333343,0.333331,0.333326
6846,2012-06-27 23:40:00,0.333466,0.333269,0.333266
6847,2012-06-27 23:45:00,0.333342,0.333334,0.333323
6848,2012-06-27 23:50:00,0.333334,0.333334,0.333333


## for 772167

In [65]:
pred_straight=yhat[:,43,0].detach().cpu()
#773939
pred_right=yhat[:,9,0].detach().cpu()
#7680
pred_left=yhat[:,104,0].detach().cpu()

pred_out=yhat[:,197,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-6850:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'772167'+'_metr_la.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2012-06-04 05:10:00,0.333333,0.333333,0.333333
1,2012-06-04 05:15:00,0.333333,0.333333,0.333333
2,2012-06-04 05:20:00,0.333333,0.333333,0.333333
3,2012-06-04 05:25:00,0.333333,0.333333,0.333333
4,2012-06-04 05:30:00,0.333333,0.333333,0.333333
...,...,...,...,...
6845,2012-06-27 23:35:00,0.333333,0.333333,0.333333
6846,2012-06-27 23:40:00,0.333333,0.333333,0.333333
6847,2012-06-27 23:45:00,0.333333,0.333333,0.333333
6848,2012-06-27 23:50:00,0.333333,0.333333,0.333333


##  save speed

In [71]:
import h5py
file='data/METR-LA/metr-la.h5'
 
f=h5py.File(file,'r')
f.keys()

<KeysViewHDF5 ['df']>

In [73]:
f['df']['axis0'][:]

array([b'773869', b'767541', b'767542', b'717447', b'717446', b'717445',
       b'773062', b'767620', b'737529', b'717816', b'765604', b'767471',
       b'716339', b'773906', b'765273', b'716331', b'771667', b'716337',
       b'769953', b'769402', b'769403', b'769819', b'769405', b'716941',
       b'717578', b'716960', b'717804', b'767572', b'767573', b'773012',
       b'773013', b'764424', b'769388', b'716328', b'717819', b'769941',
       b'760987', b'718204', b'718045', b'769418', b'768066', b'772140',
       b'773927', b'760024', b'774012', b'774011', b'767609', b'769359',
       b'760650', b'716956', b'769831', b'761604', b'717495', b'716554',
       b'773953', b'767470', b'716955', b'764949', b'773954', b'767366',
       b'769444', b'773939', b'774067', b'769443', b'767750', b'767751',
       b'767610', b'773880', b'764766', b'717497', b'717490', b'717491',
       b'717492', b'717493', b'765176', b'717498', b'717499', b'765171',
       b'718064', b'718066', b'765164', b'769431', 

In [83]:
decoded_array = [item.decode('utf-8') for item in f['df']['axis0'][:]]

In [87]:
pd.DataFrame(yhat[:,:,0].detach().cpu().numpy(),
            index=datetimes[-6850:],
            columns=decoded_array).to_csv('METRLA_res/speed.csv')
pd.read_csv('METRLA_res/speed.csv')

,Unnamed: 0,773869,767541,767542,717447,717446,717445,773062,767620,737529,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
0,2012-06-04 05:10:00,0.478925,0.418554,0.646751,0.248850,0.553726,0.139851,0.514454,0.453001,0.414266,...,-0.743103,0.554504,0.023485,0.422945,0.395744,0.330284,0.550453,0.518105,0.484738,0.404027
1,2012-06-04 05:15:00,0.467055,0.374632,0.627420,0.240687,0.553900,0.104847,0.471906,0.425108,0.337702,...,-0.715190,0.581365,0.080822,0.435354,0.365159,0.314266,0.550604,0.512570,0.512874,0.382916
2,2012-06-04 05:20:00,0.488794,0.377633,0.628332,0.203230,0.539096,0.066809,0.520083,0.405254,0.404880,...,-0.693751,0.578518,0.088868,0.423697,0.417253,0.319340,0.499237,0.482180,0.511788,0.434965
3,2012-06-04 05:25:00,0.480156,0.394224,0.650339,0.174786,0.577698,-0.009946,0.547044,0.408411,0.401991,...,-0.647591,0.569808,0.158521,0.434028,0.450133,0.293006,0.553596,0.500654,0.492121,0.446426
4,2012-06-04 05:30:00,0.478230,0.339073,0.625608,0.183830,0.571253,0.033267,0.525846,0.430451,0.390690,...,-0.618153,0.596593,0.147207,0.459017,0.486877,0.398081,0.600557,0.541169,0.487263,0.435508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6845,2012-06-27 23:35:00,0.603624,0.594190,0.735177,0.135115,0.230862,0.025351,0.325796,0.538200,0.429002,...,-0.384340,0.724317,0.444515,0.477545,0.667511,0.486353,0.696298,0.569608,0.679103,0.441278
6846,2012-06-27 23:40:00,0.592839,0.566341,0.711003,-0.048294,0.110265,0.034345,0.230156,0.532738,0.339066,...,-0.416947,0.720938,0.436690,0.422162,0.619358,0.406180,0.681215,0.551438,0.638085,0.390964
6847,2012-06-27 23:45:00,0.607599,0.589974,0.715965,0.080926,0.277033,0.076008,0.365962,0.521530,0.452204,...,-0.316797,0.724534,0.461390,0.469257,0.655949,0.473701,0.702140,0.575953,0.665389,0.434828
6848,2012-06-27 23:50:00,0.613024,0.606626,0.707457,-0.196750,-0.207318,-0.124871,0.212106,0.511172,0.032620,...,-0.869938,0.743459,0.364476,0.515784,0.608562,0.434733,0.767309,0.553631,0.672679,0.389775
